In [ ]:
# CELL 0 — Imports, constants, cosmology

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

# Physical constants
G    = 6.67430e-11        # m^3 kg^-1 s^-2
c    = 2.99792458e8       # m/s
M_sun = 1.98847e30        # kg
kpc   = 3.085677581e19    # m

# Cosmology (same as J1206 notebook)
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# ================================================================
# CELL 1 — Lens configuration input block for PG 1115+080
# ================================================================
# Change ONLY this cell for each lens.

# 1. Redshifts (lens & source)
z_d = 0.310     # lens galaxy redshift
z_s = 1.722     # quasar source redshift

# 2. Angular-diameter distances using CELL 0 cosmology
D_d  = cosmo.angular_diameter_distance(z_d)            # [Mpc]
D_s  = cosmo.angular_diameter_distance(z_s)            # [Mpc]
D_ds = cosmo.angular_diameter_distance_z1z2(z_d, z_s)  # [Mpc]

# --- NEW PART: convert to metres and build time-delay distance factor ---
D_d_m  = D_d.to(u.m).value
D_s_m  = D_s.to(u.m).value
D_ds_m = D_ds.to(u.m).value

# Time-delay distance factor: (1+z_d) D_d D_s / D_ds   [metres]
time_delay_distance_factor = (1.0 + z_d) * (D_d_m * D_s_m / D_ds_m)

print(f"(1+z_d) D_d D_s / D_ds = {time_delay_distance_factor:.3e} m")

# 3. Einstein radius (arcsec)
theta_E_arcsec = 1.135

# 4. Lens mass ellipticity and orientation
axis_ratio_q = 0.82          # ellipticity of the main lens
position_angle_phi = 68.0    # degrees East of North

# 5. External shear (from H0LiCOW PG1115 model)
gamma_ext     = 0.097        # external shear amplitude
phi_gamma_deg = 57.0         # shear angle in degrees

# 6. External convergence (LOS mass-sheet)
kappa_env = 0.075

# 7. Image positions (arcsec, lens-centered)
image_positions = {
    "A1": ( +1.153,  -0.725 ),
    "A2": ( +1.241,  -0.341 ),
    "B" : ( -0.344,  -1.127 ),
    "C" : ( -0.830,  +1.059 )
}

# 8. MGE parameters (shape only)
MGE_sigmas_arcsec = [
    0.06, 0.12, 0.20, 0.35, 0.55,
    0.80, 1.15, 1.70, 2.40, 3.80
]

MGE_amps = [
    7.5, 6.0, 4.8, 3.8, 2.5,
    1.6, 1.0, 0.6, 0.3, 0.12
]

(1+z_d) D_d D_s / D_ds = 5.127e+25 m


In [ ]:
# CELL 2 — Build grid & MGE model (PG 1115+080)

arcsec_to_rad = (np.pi/180)/3600

x_vals_arcsec = np.linspace(-5, 5, 500)
y_vals_arcsec = np.linspace(-5, 5, 500)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp*x + sinp*y
    y_rot = -sinp*x + cosp*y
    r2 = x_rot**2 + (y_rot/q)**2
    return amp*np.exp(-0.5*r2/sigma**2)

def psi_norm(x, y):
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

# Base MGE potential shape (no extra slope tweak for now)
psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# CELL 5 — Interpolate ψ at image positions and compute Δt

def psi_at(theta_x, theta_y):
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# ----- Choose which image pair you want the delay for -----
image_A_label = "A1"   # first image
image_B_label = "C"    # second image (change to "B" for A1–B)

# Extract real images
psiA = psi_at(*image_positions[image_A_label])
psiB = psi_at(*image_positions[image_B_label])

delta_psi = abs(psiB - psiA)

# SFH time delay in days
delta_t_raw = (delta_psi * time_delay_distance_factor / c) / (24 * 3600)
delta_t_corr = delta_t_raw / (1 - kappa_env)

print(f"ψ_{image_A_label} = {psiA:.6e}")
print(f"ψ_{image_B_label} = {psiB:.6e}")
print(f"Δψ ({image_B_label}-{image_A_label}) = {delta_psi:.6e}")
print(f"Raw SFH Δt ({image_B_label}-{image_A_label}) = {delta_t_raw:.2f} days")
print(f"κ_env-corrected Δt ({image_B_label}-{image_A_label}) = {delta_t_corr:.2f} days")

ψ_A1 = 1.008961e-11
ψ_C = 1.167279e-11
Δψ (C-A1) = 1.583182e-12
Raw SFH Δt (C-A1) = 3.13 days
κ_env-corrected Δt (C-A1) = 3.39 days
